These are the necessary libraries. (you may need to install matplot and networks using pip or any python package manager of your choosing)

In [ ]:
import random
import networkx as nx
import matplotlib.pyplot as plt
import copy
from collections import deque

This code defines a Python class called Graph for creating and manipulating graphs. The class uses the NetworkX library for graph operations. The class includes methods for adding vertices and edges, changing vertex colors, drawing the graph, generating random graphs, and performing graph search algorithms such as Breadth-First Search (BFS) and Depth-First Search (DFS). It also includes methods for checking cycles and performing topological sort on directed acyclic graphs.

In [ ]:
class Graph:
    SUPPORTED_TYPES = ['directed', 'undirected']
    EDGE_COLOR = 'black'
    VISITED_VERTEX_COLOR = 'yellow'
    VERTEX_COLOR = 'aqua'
    GOAL_VERTEX_COLOR = 'green'
    FRINGE_VERTEX_COLOR = 'orange'
    EXPLORED_VERTEX_COLOR = 'red'
    VERTEX_SIZE = 300

    def __init__(self,graphType):
        if graphType not in self.SUPPORTED_TYPES:
            raise Exception(f'Unsupported graph type: {type}')
        self.type=graphType
        self.graph = nx.DiGraph() if self.type == 'directed' else nx.Graph()
        self.layout = None
    def addVertex(self, vertexId):
        self.graph.add_node(vertexId,color=Graph.VERTEX_COLOR, size=Graph.VERTEX_SIZE)
    def addEdge(self, vertexId1, vertexId2):
        self.graph.add_edge(vertexId1, vertexId2, color=Graph.EDGE_COLOR)
    def addEdgeList(self, edgeList):
        self.graph.add_edges_from(edgeList)
    def changeVertexColor(self, vertexId, color):
        self.graph.nodes[vertexId]['color'] = color
    def draw(self,title):
        if self.layout is None:
            self.layout=nx.spring_layout(self.graph)

        node_colors=[color for color in nx.get_node_attributes(self.graph,'color').values()]
        nx.draw(self.graph, self.layout, with_labels=True, node_color=node_colors, edge_color=Graph.EDGE_COLOR, node_size=Graph.VERTEX_SIZE)
        plt.title(title)
        plt.show()

    def generateRandomGraph(self, vertexCount, edgeCount):
        if self.type=='directed' and edgeCount>vertexCount*(vertexCount-1):
            raise Exception(f'Number of edges should be less than {edgeCount>vertexCount*(vertexCount-1)} for directed graph')
        elif self.type=='undirected' and edgeCount>vertexCount*(vertexCount-1)/2:
            raise Exception(f'Number of edges should be less than {edgeCount>vertexCount*(vertexCount-1)/2} for undirected graph')
        
        for i in range(1,vertexCount+1):
            self.addVertex(i)
        
        if self.type=='directed':
            all_edges=[[i,j] for i in range(1,vertexCount+1) for j in range(1,vertexCount+1) if i!=j]
        elif self.type=='undirected':
            all_edges=[[i,j] for i in range(1,vertexCount+1) for j in range(i+1,vertexCount+1)]

        edges=random.sample(all_edges,edgeCount)

        for edge in edges:
            self.addEdge(edge[0],edge[1])

        self.draw(f'Randomly generated graph with {vertexCount} vertices and {edgeCount} edges')

        
    def bfs(self, source, goal):
        tempGraph=copy.deepcopy(self)
        
        iteration=0
        tempGraph.draw(f'Iteration {iteration} source is {source} goal is {goal}')

        queue=deque()
        visited=set()

        queue.append(source)
        visited.add(source)
        while len(queue):
            iteration+=1
            current=queue.popleft()

            if current not in visited:
                visited.add(current)
                tempGraph.changeVertexColor(current, Graph.VISITED_VERTEX_COLOR)
        
            
            tempGraph.draw(f'Start of Iteration {iteration} current is {current} goal is {goal}')

            if current==goal:
                tempGraph.changeVertexColor(current, Graph.GOAL_VERTEX_COLOR)
                tempGraph.draw(f'Found the Goal! At Iteration {iteration}')
                return True
            
            for neighbor in self.graph.neighbors(current):
                if neighbor not in visited:
                    queue.append(neighbor)
                    tempGraph.changeVertexColor(neighbor, Graph.FRINGE_VERTEX_COLOR)
            tempGraph.changeVertexColor(current, Graph.EXPLORED_VERTEX_COLOR)
            tempGraph.draw(f'End of Iteration {iteration} current is {current} goal is {goal}')
        
        tempGraph.draw(f'Goal not found! Final graph')

    def dfs(self, source, goal):
        tempGraph=copy.deepcopy(self)
        
        iteration=0
        tempGraph.draw(f'Iteration {iteration} source is {source} goal is {goal}')

        stack=deque()
        visited=set()

        stack.append(source)
        visited.add(source)
        while len(stack):
            iteration+=1
            current=stack.pop()

            if current not in visited:
                tempGraph.changeVertexColor(current, Graph.VISITED_VERTEX_COLOR)
                visited.add(current)

            tempGraph.draw(f'Start of Iteration {iteration} current is {current} goal is {goal}')

            if current==goal:
                tempGraph.changeVertexColor(current, Graph.GOAL_VERTEX_COLOR)
                tempGraph.draw(f'Found the Goal! Iteration {iteration}')
                return True
          
            for neighbor in self.graph.neighbors(current):
                if neighbor not in visited and neighbor not in stack:
                    stack.append(neighbor)
                    tempGraph.changeVertexColor(neighbor, Graph.FRINGE_VERTEX_COLOR)
            tempGraph.changeVertexColor(current, Graph.EXPLORED_VERTEX_COLOR)
            tempGraph.draw(f'End of Iteration {iteration} current is {current} goal is {goal}')
        
        tempGraph.draw(f'Goal not found! Final graph')

    def checkCycleDfsUtil(self, vertex, visited, recStack):
        visited.add(vertex)
        recStack.add(vertex)

        for neighbor in self.graph.neighbors(vertex):
            if neighbor not in visited:
                if self.checkCycleDfsUtil(neighbor, visited, recStack):
                    return True
            elif neighbor in recStack:
                return True
    
        recStack.remove(vertex)
        return False
    
    def checkCycleDfs(self):
        visited=set()
        recStack=set()

        for vertex in self.graph.nodes():
            if vertex not in visited:
                if self.checkCycleDfsUtil(vertex, visited, recStack):
                    return True
        return False

    def topologicalSortUtil(self,vertex,visited,stack):
        visited.add(vertex)

        for neighbor in self.graph.neighbors(vertex):
            if neighbor not in visited:
                self.topologicalSortUtil(neighbor, visited, stack)
        
        stack.append(vertex)
        

    def topologicalSort(self):
        if self.type!='directed':
            raise Exception(f'Topological sort is only supported for directed graphs')

        if  self.checkCycleDfs():
            raise Exception(f'Topological sort is only supported for directed acyclic graphs')

        self.draw(f'Graph for topological sort')

        stack=deque()
        visited=set()

        for vertex in self.graph.nodes():
            if vertex not in visited:
                self.topologicalSortUtil(vertex, visited, stack)

        stack.reverse()
        i=0
        for vertex in stack:
            self.changeVertexColor(vertex, Graph.VISITED_VERTEX_COLOR)
            self.draw(f'The {i}th vertex in topological sort is {vertex}')
            i+=1

Below are some examples on how to use the Graph class. For more information look at slides.

In [ ]:
# # Create a directed graph
# myGraph = Graph('directed')

# # Add vertices
# myGraph.addVertex(1)
# myGraph.addVertex(2)
# myGraph.addVertex(3)

# # Add edges
# myGraph.addEdge(1, 2)
# myGraph.addEdge(2, 3)
# myGraph.addEdge(3, 1)

# # Draw the graph
# myGraph.draw("Directed Graph Example")

# # Create an undirected graph
# randomGraph = Graph('undirected')

# # Generate a random graph with 5 vertices and 7 edges
# randomGraph.generateRandomGraph(5, 7)

# # Perform Breadth-First Search
# randomGraph.bfs(1, 5)

# # Create a directed graph
# myDirectedGraph = Graph('directed')

# # Add vertices and edges
# myDirectedGraph.addEdgeList([(1, 2), (2, 3), (3, 4), (4, 1)])

# # Perform Depth-First Search
# myDirectedGraph.dfs(1, 4)

# # Create a directed graph with a cycle
# cycleGraph = Graph('directed')
# cycleGraph.addEdgeList([(1, 2), (2, 3), (3, 1)])

# # Check for cycles
# hasCycle = cycleGraph.checkCycleDfs()
# print(f"Graph has a cycle: {hasCycle}")

# # Create a directed acyclic graph
# dag = Graph('directed')
# dag.addEdgeList([(1, 2), (1, 3), (2, 4), (3, 4)])

# # Perform Topological Sort
# dag.topologicalSort()